## 파일 불러오기

In [60]:
import pandas as pd
import glob
import os  

base_path = '/Users/apstat/Desktop/연구/멀티모달 밸런싱/데이터'
search_pattern = os.path.join(base_path, "Sess*.csv")
file_paths = glob.glob(search_pattern)

print(f"총 {len(file_paths)}개의 파일을 찾았습니다:")
# print(file_paths)

all_dfs = []
for path in file_paths:
    try:
        all_dfs.append(pd.read_csv(path))
    except Exception as e:
        print(f"파일을 읽는 중 오류 발생: {path}, 오류: {e}")


if all_dfs:
    df_all = pd.concat(all_dfs, ignore_index=True)
    print(f"\n데이터 통합 완료. 총 {len(df_all)}개의 행이 로드되었습니다.")
    print(f"7개 감정 클래스 확인: {df_all['Emotion'].unique()}")
else:
    print("\n[오류] CSV 파일을 찾지 못했거나 읽을 수 없습니다. 경로와 파일 이름을 확인하세요.")

총 40개의 파일을 찾았습니다:

데이터 통합 완료. 총 301008개의 행이 로드되었습니다.
7개 감정 클래스 확인: ['neutral' 'happy' 'angry' 'disgust' 'sad' 'surprise' 'fear']


In [61]:
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.utils import to_categorical

# 1a. 특징 및 레이블 정의
# 시계열 특징(EDA, TEMP)과 정적 특징(Valence, Arousal)을 모두 사용
features = ['EDA', 'TEMP', 'Valence', 'Arousal']
target = 'Emotion'

# 1b. 특징 스케일링 (StandardScaler)
# (중요!) 10개 파일의 모든 행에 대해 스케일러를 'fit'
scaler = StandardScaler()
df_all[features] = scaler.fit_transform(df_all[features])

# 1c. 레이블 인코딩
# 'happy' -> 2, 'neutral' -> 4 등 숫자로 변환
label_encoder = LabelEncoder()
df_all[target] = label_encoder.fit_transform(df_all[target])

# 7개 감정 클래스 확인
n_classes = len(label_encoder.classes_)
print(f"총 {n_classes}개의 감정 클래스를 찾았습니다: {label_encoder.classes_}")

총 7개의 감정 클래스를 찾았습니다: ['angry' 'disgust' 'fear' 'happy' 'neutral' 'sad' 'surprise']


In [62]:
df_all.head()

,Segment_ID,Time,EDA,TEMP,Emotion,Valence,Arousal
0,Sess21_script01_User042F_001,0.25,-0.056340,0.172719,4,0.94888,-0.324641
1,Sess21_script01_User042F_001,0.50,-0.057462,0.172719,4,0.94888,-0.324641
2,Sess21_script01_User042F_001,0.75,-0.061050,0.172719,4,0.94888,-0.324641
3,Sess21_script01_User042F_001,1.00,-0.061722,0.172719,4,0.94888,-0.324641
4,Sess21_script01_User042F_001,1.25,-0.061050,0.172719,4,0.94888,-0.324641


In [63]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

print("\n--- 2. 시퀀스 생성 (Padding) ---")

# 2a. Segment_ID별로 묶기
grouped = df_all.groupby('Segment_ID')

# 2b. 각 Segment_ID의 (길이, 4개 특징) 배열을 리스트에 저장
X_sequences = []
y_labels = []

for name, group in grouped:
    # 4개 특징의 시퀀스 (numpy array)
    X_sequences.append(group[features].values)
    # 해당 ID의 첫 번째 레이블 (어차피 다 같음)
    y_labels.append(group[target].iloc[0])

# 2c. 가장 긴 시퀀스의 길이(TimeSteps) 찾기
max_len = max(len(seq) for seq in X_sequences)
print(f"가장 긴 시퀀스의 길이 (max_len): {max_len}")

# 2d. 패딩(Padding) 수행
# 'post': 시퀀스 뒤쪽에 0.0을 채움
X_padded = pad_sequences(
    X_sequences, 
    maxlen=max_len, 
    padding='post', 
    dtype='float32', 
    value=0.0 # 패딩 값은 0.0
)

# 2e. 레이블을 numpy 배열로 변환 및 원-핫 인코딩
y_array = np.array(y_labels)
y_categorical = to_categorical(y_array, num_classes=n_classes)

print(f"최종 X 데이터 형태 (샘플, 최대길이, 특징): {X_padded.shape}")
print(f"최종 y 데이터 형태 (샘플, 클래스 수): {y_categorical.shape}")


--- 2. 시퀀스 생성 (Padding) ---
가장 긴 시퀀스의 길이 (max_len): 141
최종 X 데이터 형태 (샘플, 최대길이, 특징): (12763, 141, 4)
최종 y 데이터 형태 (샘플, 클래스 수): (12763, 7)


In [64]:
from sklearn.model_selection import train_test_split

print("\n--- 3. 데이터 분리 (6:2:2) ---")

# X = X_padded, y = y_categorical
X_train, X_temp, y_train, y_temp = train_test_split(
    X_padded, y_categorical,
    test_size=0.4, 
    random_state=42, 
    stratify=y_array # (중요!) 원-핫 인코딩 전의 y_array로 비율 맞춤
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.5, 
    random_state=42, 
    stratify=y_temp.argmax(axis=1) # (임시 y로 비율 맞춤)
)

print(f"Train: {X_train.shape}, {y_train.shape}")
print(f"Validation: {X_val.shape}, {y_val.shape}")
print(f"Test: {X_test.shape}, {y_test.shape}")


--- 3. 데이터 분리 (6:2:2) ---
Train: (7657, 141, 4), (7657, 7)
Validation: (2553, 141, 4), (2553, 7)
Test: (2553, 141, 4), (2553, 7)


## simple RNN

In [65]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, Masking
from tensorflow.keras.layers import SimpleRNN  # [수정] LSTM 대신 SimpleRNN을 import
from tensorflow.keras.callbacks import EarlyStopping

print("\n--- 4. 기본 RNN (SimpleRNN) 모델 구축 ---")

# 4a. 모델 파라미터 정의
n_features = 4  # (EDA, TEMP, Valence, Arousal)
# max_len은 2단계에서 계산됨

# 4b. 모델 생성
rnn_model = Sequential()

# Input Layer 및 Masking Layer (동일)
rnn_model.add(Input(shape=(max_len, n_features)))
rnn_model.add(Masking(mask_value=0.0))

# [수정] LSTM 레이어 대신 SimpleRNN 레이어 사용
# (SimpleRNN은 vanishing gradient 문제로 LSTM보다 성능이 낮을 수 있음)
rnn_model.add(SimpleRNN(units=64))

# (선택) 과적합 방지 (동일)
rnn_model.add(Dropout(0.3))
rnn_model.add(Dense(units=32, activation='relu'))

# Output Layer (동일)
rnn_model.add(Dense(units=n_classes, activation='softmax'))

# 4c. 모델 컴파일 (동일)
rnn_model.compile(
    optimizer='adam', 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

rnn_model.summary()


--- 4. 기본 RNN (SimpleRNN) 모델 구축 ---


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking_9 (Masking)             │ (None, 141, 4)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_6 (SimpleRNN)        │ (None, 64)             │         4,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 7)              │           231 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,727 (26.28 KB)

 Trainable params: 6,727 (26.28 KB)

 Non-trainable params: 0 (0.00 B)

In [66]:
from sklearn.metrics import classification_report

print("\n--- 5. 모델 학습 및 평가 ---")

# 5a. 조기 종료(EarlyStopping) 설정
# Validation loss가 5번 연속 개선되지 않으면 학습 중지
early_stopper = EarlyStopping(
    monitor='val_loss', 
    patience=5, 
    restore_best_weights=True # 가장 좋았던 가중치 복원
)

# 5b. 모델 학습
history = rnn_model.fit(
    X_train, y_train,
    epochs=50, # (최대 50번, 그 전에 조기 종료될 수 있음)
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[early_stopper]
)

print("\n학습 완료.")


--- 5. 모델 학습 및 평가 ---
Epoch 1/50
240/240 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8570 - loss: 0.5063 - val_accuracy: 0.9080 - val_loss: 0.3349
Epoch 2/50
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9051 - loss: 0.3411 - val_accuracy: 0.9064 - val_loss: 0.3224
Epoch 3/50
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9081 - loss: 0.3230 - val_accuracy: 0.9001 - val_loss: 0.3474
Epoch 4/50
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9105 - loss: 0.3201 - val_accuracy: 0.9146 - val_loss: 0.3082
Epoch 5/50
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9118 - loss: 0.3125 - val_accuracy: 0.9130 - val_loss: 0.3036
Epoch 6/50
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9122 - loss: 0.3110 - val_accuracy: 0.9138 - val_loss: 0.3066
Epoch 7/50
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9088 - loss: 0.3080 - val_accuracy: 0.9091 - val_loss: 0.3093
Epoch 8/50
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9115 - loss: 0.

In [67]:
# 5c. 최종 평가 (Test Set)
print("\n--- 최종 평가 (Test Set) ---")
test_loss, test_accuracy = rnn_model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# 5d. 분류 리포트 (Classification Report)
y_pred_probs = rnn_model.predict(X_test)
y_pred_classes = np.argmax(y_pred_probs, axis=1) # (확률 -> 클래스 0~6)
y_test_classes = np.argmax(y_test, axis=1)     # (원-핫 -> 클래스 0~6)

# 레이블 인코더로 원래 감정 이름 가져오기
target_names = label_encoder.classes_
print("\nFinal Classification Report (Test Set):")
print(classification_report(y_test_classes, y_pred_classes, target_names=target_names, digits=4, zero_division=0))


--- 최종 평가 (Test Set) ---
Test Loss: 0.2925
Test Accuracy: 0.9146
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

Final Classification Report (Test Set):
              precision    recall  f1-score   support

       angry     1.0000    0.1724    0.2941        29
     disgust     0.0000    0.0000    0.0000        12
        fear     0.0000    0.0000    0.0000         8
       happy     0.7868    0.6596    0.7176       235
     neutral     0.9251    0.9824    0.9529      2214
         sad     0.0000    0.0000    0.0000        24
    surprise     0.0000    0.0000    0.0000        31

    accuracy                         0.9146      2553
   macro avg     0.3874    0.2592    0.2807      2553
weighted avg     0.8861    0.9146    0.8958      2553



## LSTM

In [68]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Masking
from tensorflow.keras.callbacks import EarlyStopping

print("\n--- 4. LSTM 모델 구축 ---")

# 4a. 모델 파라미터 정의
n_features = 4  # (EDA, TEMP, Valence, Arousal)
# max_len은 2단계에서 계산됨

# 4b. 모델 생성
lstm = Sequential()

# (중요!) Input Layer: 입력 형태 정의
# (None, n_features)는 (max_len, n_features)와 동일
lstm.add(Input(shape=(max_len, n_features)))

# (중요!) Masking Layer: 패딩 값 0.0을 무시하도록 설정
lstm.add(Masking(mask_value=0.0))

# LSTM Layer: 64개 유닛. 
# return_sequences=False (기본값): 마지막 타임스텝의 출력만 전달
lstm.add(LSTM(units=64))

# (선택) 과적합 방지
lstm.add(Dropout(0.3))
lstm.add(Dense(units=32, activation='relu'))

# Output Layer: 7개 감정 클래스로 분류
lstm.add(Dense(units=n_classes, activation='softmax'))

# 4c. 모델 컴파일
lstm.compile(
    optimizer='adam', 
    loss='categorical_crossentropy', # (원-핫 인코딩이므로)
    metrics=['accuracy']
)

lstm.summary()


--- 4. LSTM 모델 구축 ---


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking_10 (Masking)            │ (None, 141, 4)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 64)             │        17,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 7)              │           231 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,975 (78.03 KB)

 Trainable params: 19,975 (78.03 KB)

 Non-trainable params: 0 (0.00 B)

In [69]:
from sklearn.metrics import classification_report

print("\n--- 5. 모델 학습 및 평가 ---")

# 5a. 조기 종료(EarlyStopping) 설정
# Validation loss가 5번 연속 개선되지 않으면 학습 중지
early_stopper = EarlyStopping(
    monitor='val_loss', 
    patience=5, 
    restore_best_weights=True # 가장 좋았던 가중치 복원
)

# 5b. 모델 학습
history = lstm.fit(
    X_train, y_train,
    epochs=50, # (최대 50번, 그 전에 조기 종료될 수 있음)
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[early_stopper]
)

print("\n학습 완료.")


--- 5. 모델 학습 및 평가 ---
Epoch 1/50
240/240 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.8643 - loss: 0.5561 - val_accuracy: 0.9119 - val_loss: 0.3303
Epoch 2/50
240/240 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.8960 - loss: 0.3434 - val_accuracy: 0.9099 - val_loss: 0.3190
Epoch 3/50
240/240 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.9030 - loss: 0.3262 - val_accuracy: 0.9142 - val_loss: 0.3094
Epoch 4/50
240/240 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.9116 - loss: 0.3106 - val_accuracy: 0.9150 - val_loss: 0.2994
Epoch 5/50
240/240 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.9113 - loss: 0.3063 - val_accuracy: 0.9134 - val_loss: 0.2953
Epoch 6/50
240/240 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.9134 - loss: 0.3006 - val_accuracy: 0.9115 - val_loss: 0.3044
Epoch 7/50
240/240 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.9146 - loss: 0.3015 - val_accuracy: 0.9138 - val_loss: 0.2952
Epoch 8/50
240/240 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.9122 - 

In [70]:
# 5c. 최종 평가 (Test Set)
print("\n--- 최종 평가 (Test Set) ---")
test_loss, test_accuracy = lstm.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# 5d. 분류 리포트 (Classification Report)
y_pred_probs = lstm.predict(X_test)
y_pred_classes = np.argmax(y_pred_probs, axis=1) # (확률 -> 클래스 0~6)
y_test_classes = np.argmax(y_test, axis=1)     # (원-핫 -> 클래스 0~6)

# 레이블 인코더로 원래 감정 이름 가져오기
target_names = label_encoder.classes_
print("\nFinal Classification Report (Test Set):")
print(classification_report(y_test_classes, y_pred_classes, target_names=target_names, digits=4, zero_division=0))


--- 최종 평가 (Test Set) ---
Test Loss: 0.2927
Test Accuracy: 0.9142
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step

Final Classification Report (Test Set):
              precision    recall  f1-score   support

       angry     0.6250    0.1724    0.2703        29
     disgust     0.0000    0.0000    0.0000        12
        fear     0.0000    0.0000    0.0000         8
       happy     0.7979    0.6553    0.7196       235
     neutral     0.9247    0.9824    0.9527      2214
         sad     0.0000    0.0000    0.0000        24
    surprise     0.0000    0.0000    0.0000        31

    accuracy                         0.9142      2553
   macro avg     0.3354    0.2586    0.2775      2553
weighted avg     0.8825    0.9142    0.8955      2553

